# 在 Colab 跑 OWOBJ 🛠️
從掛載 Google Drive、clone、安裝依賴與 CUDA 編譯，到資料放置與單卡訓練。

### 0. 檢查 GPU（務必使用 GPU runtime）
- Runtime > Change runtime type > GPU

In [ ]:
!nvidia-smi

### 1. 掛載雲端硬碟並設定路徑
- `DATA_ROOT` 用來放 VOC 風格的 OWOD 資料，存在 Drive 才能持久化。
- `COCO_ROOT` 可選，先暫存 COCO 再轉成 VOC XML。

In [ ]:
from google.colab import drive
import os, pathlib
drive.mount('/content/drive')

PROJECT_ROOT = '/content/owobj'
DRIVE_ROOT = '/content/drive/MyDrive'
DATA_ROOT = f"{DRIVE_ROOT}/datasets/OWOD"
COCO_ROOT = f"{DRIVE_ROOT}/datasets/coco"  # optional staging area before conversion

for p in [DATA_ROOT, COCO_ROOT]:
    pathlib.Path(p).mkdir(parents=True, exist_ok=True)

# export for bash cells
os.environ['PROJECT_ROOT'] = PROJECT_ROOT
os.environ['DATA_ROOT'] = DATA_ROOT
os.environ['COCO_ROOT'] = COCO_ROOT

print('PROJECT_ROOT:', PROJECT_ROOT)
print('DATA_ROOT:', DATA_ROOT)
print('COCO_ROOT:', COCO_ROOT)

### 2. 下載或更新專案

In [ ]:
import os
if not os.path.exists(PROJECT_ROOT):
    !git clone https://github.com/AI4Math-ShanZhang/OWOBJ.git $PROJECT_ROOT
else:
    %cd $PROJECT_ROOT
    !git pull
%cd $PROJECT_ROOT

### 3. 安裝 Python 依賴與 PyTorch（CUDA 11.3 版本）
- 先重裝 torch 避免後續編譯 CUDA op 時 ABI 不相容。

In [ ]:
%cd $PROJECT_ROOT
!pip install -q torch==1.12.0+cu113 torchvision==0.13.0+cu113 torchaudio==0.12.0 --extra-index-url https://download.pytorch.org/whl/cu113
!pip install -q -r requirements.txt ninja

### 4. 編譯 Deformable-DETR CUDA operators
- 訓練前必做；`test.py` 是簡易檢查。

In [ ]:
%cd $PROJECT_ROOT/models/ops
!python setup.py build install
!python test.py
%cd $PROJECT_ROOT

### 5. 下載 backbone 權重
- DINO ResNet-50 放在 `models/`。

In [ ]:
%cd $PROJECT_ROOT
!mkdir -p models
!wget -c https://dl.fbaipublicfiles.com/dino/dino_resnet50_pretrain/dino_resnet50_pretrain.pth -O models/dino_resnet50_pretrain.pth

### 6. 資料夾結構與準備方式
**目標 (VOC 風格)：**
```
OWOBJ/
└── data/
    ├── OWOD/
    │   ├── JPEGImages/           # 所有圖片 (COCO + VOC)
    │   ├── Annotations/          # VOC XML (來自 COCO + VOC)
    │   └── ImageSets/
    │       ├── OWDETR/
    │       ├── TOWOD/
    │       └── VOC2007/
    └── coco/                     # 可選，用來暫存原始 COCO
        ├── annotations/
        ├── train2017/
        └── val2017/
```
佈署步驟：
1) **COCO 2017**：把 `train2017/`、`val2017/`、`annotations/*.json` 放到 `$COCO_ROOT`（或 `data/coco/`）。
2) **COCO -> VOC XML**：執行 `datasets/coco2voc.py`，會在 `data/OWOD/Annotations/` 產生 XML，並寫入 `ImageSets/train.txt`。
3) **PASCAL VOC 2007+2012**：解壓 trainval/test，把圖片放入 `data/OWOD/JPEGImages/`，XML 放入 `data/OWOD/Annotations/`。
4) 確認 `ImageSets/OWDETR`、`ImageSets/TOWOD`、`ImageSets/VOC2007` 都在（專案已附，請與資料放一起）。
建議：真實資料放在 Drive (`DATA_ROOT`)，再用 symlink 連到 `data/OWOD`，每次開 Colab 就不用重下載。

In [ ]:
%cd $PROJECT_ROOT
import pathlib
pathlib.Path('data').mkdir(exist_ok=True)
owod_link = pathlib.Path('data/OWOD')
if not owod_link.exists():
    owod_link.symlink_to(pathlib.Path(DATA_ROOT), target_is_directory=True)
    print('已建立 data/OWOD ->', DATA_ROOT)
else:
    print('data/OWOD 已存在，未變動。')

coco_link = pathlib.Path('data/coco')
if not coco_link.exists():
    coco_link.symlink_to(pathlib.Path(COCO_ROOT), target_is_directory=True)
    print('已建立 data/coco ->', COCO_ROOT)
else:
    print('data/coco 已存在，未變動。')

### 7. （選擇性）下載 COCO 2017 到雲端硬碟
已有就跳過；檔案約 25GB。

In [ ]:
%cd $COCO_ROOT
# 若需要下載，解除下列註解（耗時、占空間）
# !wget -c http://images.cocodataset.org/zips/train2017.zip
# !wget -c http://images.cocodataset.org/zips/val2017.zip
# !wget -c http://images.cocodataset.org/annotations/annotations_trainval2017.zip
# !unzip -q train2017.zip
# !unzip -q val2017.zip
# !unzip -q annotations_trainval2017.zip

### 8. COCO 轉 VOC XML 供 OWOD 使用
先確保 `data/coco` 已有 COCO 檔。

In [ ]:
%cd $PROJECT_ROOT
!python datasets/coco2voc.py

### 9. 單卡訓練（示範）
- 取自 `configs/M_OWOD_BENCHMARK.sh` 的 task 1，batch 調小以適應單卡。
- OOM 就再降低 `--batch_size`。模型與 log 存在 `exps/MOWODB/OWOBJ_colab/t1_single/`。

In [ ]:
%%bash
set -e
cd "$PROJECT_ROOT"
WANDB_NAME=owobj_colab  # exemplar replay 會用到此名稱
CUDA_VISIBLE_DEVICES=0 python main_open_world.py \
  --output_dir "exps/MOWODB/OWOBJ_colab/t1_single" \
  --dataset OWDETR --PREV_INTRODUCED_CLS 0 --CUR_INTRODUCED_CLS 19 \
  --train_set 't1_train' --test_set 'test' --epochs 41 \
  --cls_loss_coef 2 --focal_alpha 0.25 \
  --model_type 'sketch' --obj_loss_coef 8e-4 --obj_temp 1.3 --obj_kl_div 0.1 \
  --exemplar_replay_max_length 850 \
  --exemplar_replay_dir "$WANDB_NAME" --exemplar_replay_cur_file "learned_owod_t1_ft.txt" \
  --batch_size 2 --num_workers 4 \
  --device cuda

### 10. 備註
- checkpoint/log 在 `exps/`；如需持久化可把 `exps` 也 symlink 到 Drive。
- t2–t4 依照 `configs/M_OWOD_BENCHMARK.sh` 調整，並用上一階段的 `--pretrain`。
- 評估：`bash run_eval.sh`（需對應 checkpoint 放在 `exps/` 結構）。